In [17]:
import secrets
import subprocess
import tempfile

from hammers.osapi import Auth, load_osrc
from hammers.osrest import glance, keystone

In [18]:
rc = {
    'tacc': load_osrc('../rc/admin-chi.tacc.rc', get_pass=True),
    'uc': load_osrc('../rc/admin-chi.uc.rc'),
    'kvm': load_osrc('../rc/admin-kvm.tacc.rc'),
}
rc['uc']['OS_PASSWORD'] = rc['kvm']['OS_PASSWORD'] = rc['tacc']['OS_PASSWORD']
auth = {k: Auth(rc[k]) for k in rc}

Enter your password: ········


In [19]:
BASE_NAME = {
    'ubuntu-trusty': 'Ubuntu14.04',
    'ubuntu-xenial': 'Ubuntu16.04',
    'centos7': 'CentOS7',
}
VARIANT_NAME = {
    'base': '',
    'gpu': 'CUDA8',
    'fpga': 'FPGA',
}

def production_name(image=None, os=None, variant=None):
    if os is None and variant is None:
        if image is None:
            raise ValueError('must provide image or os/variant')
        os = image['build-os']
        variant = image['build-variant']
    elif os is None or variant is None:
        raise ValueError('must provide image or os/variant')

    base = BASE_NAME[os]
    variant = VARIANT_NAME[variant]
    var_delim = '-' if variant else ''
    return 'CC-{}{}{}'.format(base, var_delim, variant)


def archival_name(image=None, os=None, variant=None):
    return '{}-{}'.format(production_name(image, os, variant), image['build-os-base-image-revision'])


BASE_PROPS = {
    'checksum',
    'container_format',
    'created_at',
    'disk_format',
    'file',
    'id',
    'min_disk',
    'min_ram',
    'name',
    'owner',
    'protected',
    'schema',
    'self',
    'size',
    'status',
    'tags',
    'updated_at',
    'virtual_size',
    'visibility',
}

def extract_extra_properties(image):
    return {k: image[k] for k in image if k not in BASE_PROPS}


class ImageDistributor:
    def __init__(self, auths, build_site, os, variant):
        self.auths = auths
        self.build_site = build_site
        self.os = os
        self.variant = variant
        
        self.production_name = production_name(os=self.os, variant=self.variant)
        
        self.work_order = None

    def current_production_image(self, site):
        # filter images
        images_with_prod_name = glance.images(self.auths[site], {
            'name': self.production_name, 
            'visibility': 'public',
        })
        if len(images_with_prod_name) > 1:
            raise RuntimeError('multiple images with production name "{}" on site "{}":\n{}'.format(
                self.production_name, 
                site, 
                '\n'.join('- {}'.format(i['id']) for i in images_with_prod_name)
            ))
        elif len(images_with_prod_name) < 1:
            return None
        
        prod_image = images_with_prod_name[0]
        if 'build-tag' not in prod_image:
            print('Warning: Metadata missing from current production image "{}" on site "{}"'.format(
                self.production_name,
                site
            ))
        return prod_image['id']
        
    def propose_work_order(self):
        orders = []
        
        # filter images
        images = glance.images(self.auths[self.build_site], {
            'build-os': self.os, 
            'build-variant': self.variant,
        })
        images = {i['id']: i for i in images if i['build-tag'].startswith('jenkins')}
        if not images:
            raise RuntimeError('no images found')
        image_order = sorted(images, key=lambda i: images[i]['build-tag'], reverse=True)
        candidate_image = images[image_order[0]]
            
        # find current published image
        published_images = {}
        for site in self.auths:
            published_images[site] = self.current_production_image(site)
#         try:
#             published_image = next(
#                 images[i] 
#                 for i 
#                 in image_order 
#                 if images[i]['visibility'] == 'public' and 
#                    images[i]['name'] == self.production_name)
#         except StopIteration:
#             images_with_prod_name = glance.images(self.auth, {
#                 'name': self.production_name, 
#                 'visibility': 'public',
#             })
#             if len(images_with_prod_name) < 1:
#                 print('Warning: No public images found matching name "{}". Assuming new release.'
#                       .format(self.production_name))
#                 published_image = None
#             elif len(images_with_prod_name) == 1:
#                 print('Info: Current production image "{}" has no metadata, matched by name'
#                       .format(self.production_name))
#                 published_image = images_with_prod_name[0]
#             else:
#                 raise RuntimeError('Multiple public images with name "{}", none have metadata'
#                                    .format(self.production_name))

        orders = []
#         if not any(published_images.values) and published_image['id'] == candidate_image['id']:
#             # nothing to do
#             pass
#         elif published_image is None:
#             orders.append(('publish', candidate_image['id']))
#         else:
#             orders.append(('archive', published_image['id']))
#             orders.append(('publish', candidate_image['id']))
        for site in published_images:
            if published_images[site] is not None:
                orders.append(('archive', (site, published_images[site])))
            orders.append(('copypublish', (self.build_site, candidate_image['id']), (site, 'id-{}'.format(site))))
    
        self.proposed_orders = orders
        return orders

    def archive(self, site, image):
        # check if anticipated archival name exists
        arch_name = archival_name(image)
        possible_clashes = glance.images(self.auths[site], {
            'build-os-base-image-revision': image['build-os-base-image-revision'],
        })
        if possible_clashes:
            # append an "-[integer]" if needed
            possible_clashes.sort(key=lambda i: i['name'], reverse=True)
            latest_archive_name = None
            for image in possible_clashes:
                if images['name'].startswith(arch_name):
                    arch_name = increment_archival_name(images)
                    break
            #else: no clashes, fall through with base arch_name
        assert not glance.images(self.auths[site], {'name': arch_name}) # should be unique
        glance.image_properties(self.auths[site], image['id'], replace={'name': arch_name})
    
    def copypublish(self, source_site, source_id, target_site):
        pass

def increment_archival_name(image):
    parts = image['name'].split('-')
    if parts[-1] == image['build-os-base-image-revision']:
        parts.append('1')
    else:
        parts[-1] = str(int(parts[-1]) + 1)
    return '-'.join(parts)

_BOBIR = 'build-os-base-image-revision'
_IMG = {'name': 'CC-CentOS7-1610.0', _BOBIR: '1610.0'}
_IMGN_2 = increment_archival_name(_IMG)
assert _IMGN_2 == 'CC-CentOS7-1610.0-1', _IMGN_2
_IMG['name'] = _IMGN_2
_IMGN_3 = increment_archival_name(_IMG)
assert _IMGN_3 == 'CC-CentOS7-1610.0-2', _IMGN_3

In [33]:
for site in auth:
    print(auth[site].access['token']['tenant']['id'])

570aad8999f7499db99eae22fe9b29bb
cdf92cfdf4c5422cb2ead2d8a8ca2ae7
openstack


In [ ]:
glance.image_properties

In [45]:
candidates = glance.images(auth['uc'], query={'build-os': 'ubuntu-xenial'})#, 'owner': auth['kvm'].access['token']['tenant']['id']})
candidates.sort(key=operator.itemgetter('created_at'), reverse=True)

In [50]:
auth['uc'].access

{'metadata': {'is_admin': 0,
  'roles': ['9fe2ff9ee4384b1894a90878d3e92bab',
   '692a231cd23a4c1a848213194d1bc3f3']},
 'serviceCatalog': [{'endpoints': [{'adminURL': 'http://10.140.80.98:8774/v2.1',
     'id': '4c14c2ed2f5749d3bb65c88337938ba5',
     'internalURL': 'http://10.140.80.98:8774/v2.1',
     'publicURL': 'https://chi.uc.chameleoncloud.org:8774/v2.1',
     'region': 'CHI@UC'}],
   'endpoints_links': [],
   'name': 'nova',
   'type': 'compute'},
  {'endpoints': [{'adminURL': 'http://10.140.80.98:9696',
     'id': '335eac699a0c4ddcaa119936dd365ec1',
     'internalURL': 'http://10.140.80.98:9696',
     'publicURL': 'https://chi.uc.chameleoncloud.org:9696',
     'region': 'CHI@UC'}],
   'endpoints_links': [],
   'name': 'neutron',
   'type': 'network'},
  {'endpoints': [{'adminURL': 'http://10.140.80.98:8774/v3',
     'id': 'b0d829f93aba40bba588efa11bd0e43f',
     'internalURL': 'http://10.140.80.98:8774/v3',
     'publicURL': 'https://chi.uc.chameleoncloud.org:8774/v3',
     're

In [49]:
s = [serv
    for serv
    in auth['uc'].access['serviceCatalog']
    if serv['type'] == 'compute'
]
s

[{'endpoints': [{'adminURL': 'http://10.140.80.98:8774/v2.1',
    'id': '4c14c2ed2f5749d3bb65c88337938ba5',
    'internalURL': 'http://10.140.80.98:8774/v2.1',
    'publicURL': 'https://chi.uc.chameleoncloud.org:8774/v2.1',
    'region': 'CHI@UC'}],
  'endpoints_links': [],
  'name': 'nova',
  'type': 'compute'}]

In [38]:
import operator

[{'build-os': 'ubuntu-xenial',
  'build-os-base-image-revision': '20170410',
  'checksum': 'c2145d34dd4b8b4d65d61fa9e1aa5c46',
  'container_format': 'bare',
  'created_at': '2016-12-23T17:33:01Z',
  'disk_format': 'qcow2',
  'file': '/v2/images/a29be6ec-cf43-488d-9fb6-9a257c96b0f6/file',
  'id': 'a29be6ec-cf43-488d-9fb6-9a257c96b0f6',
  'min_disk': 0,
  'min_ram': 0,
  'name': 'CC-Ubuntu16.04-20170410',
  'owner': 'cdf92cfdf4c5422cb2ead2d8a8ca2ae7',
  'protected': False,
  'schema': '/v2/schemas/image',
  'self': '/v2/images/a29be6ec-cf43-488d-9fb6-9a257c96b0f6',
  'size': 841482240,
  'status': 'active',
  'tags': [],
  'updated_at': '2017-11-07T05:43:35Z',
  'virtual_size': None,
  'visibility': 'public'},
 {'build-os': 'ubuntu-xenial',
  'build-os-base-image-revision': '20171028',
  'build-repo': 'https://github.com/ChameleonCloud/CC-Ubuntu16.04',
  'build-repo-commit': '0e16570cbdb11bccabb0ef3143e7e1ecf8b3724f',
  'build-tag': 'jenkins-cc-xenial-builder-10',
  'build-variant': 'bas

In [4]:
dist = ImageDistributor(auth, 'uc', 'ubuntu-xenial', 'base')
dist.propose_work_order()

[('archive', ('tacc', 'a64dec6d-54e6-4e5d-8fe6-03c939d46e96')),
 ('copypublish',
  ('uc', '7c91e2a9-4846-407f-ab1d-f2954e98fa18'),
  ('tacc', 'id-tacc')),
 ('archive', ('uc', 'b716162a-638e-4593-9587-c789a8367fdf')),
 ('copypublish',
  ('uc', '7c91e2a9-4846-407f-ab1d-f2954e98fa18'),
  ('uc', 'id-uc')),
 ('archive', ('kvm', '30adb881-f921-404d-a2de-cf530795b2aa')),
 ('copypublish',
  ('uc', '7c91e2a9-4846-407f-ab1d-f2954e98fa18'),
  ('kvm', 'id-kvm'))]

In [5]:
past_centos = {
    'tacc': {'id': '0f216b1f-7841-451b-8971-d383364e01a6'},
    'uc': {'id': '2c16bd4a-8d77-4c56-ae5d-fca651147016'},
    'kvm': {'id': 'dc59301a-8d0f-4125-a09f-8ae1965fda64'},
}
past_xenial = {
    'tacc': {'id': 'a64dec6d-54e6-4e5d-8fe6-03c939d46e96'},
    'uc': {'id': 'b716162a-638e-4593-9587-c789a8367fdf'},
    'kvm': {'id': '30adb881-f921-404d-a2de-cf530795b2aa'},
}
new_centos = {
    'tacc': {'id': '10adbb3b-ccd3-4e23-8623-b8df82073de9'},
    'uc': {'id': '388ca9a2-c7d7-49ac-8f66-6b254a6951d4'},
    'kvm': {'id': '053b3438-edb5-4d17-873f-525600b9d9a9'},
}
new_xenial = {
    'tacc': {'id': 'd930e8b3-7b29-4d25-9a63-cefdc46d6bc4'},
    'uc': {'id': '926d0ea3-6e47-40f3-87ba-596f3bbe56cc'},
    'kvm': {'id': 'c80567e8-12f9-44fc-8335-f62831ee73a6'},
}

In [52]:
for site, image in past_centos.items():
#     image['details'] = glance.image(auth[site], image['id'])
#     assert image['details']['name'] == 'CC-CentOS7'
    glance.image_properties(auth[site], image['id'], add={
        'build-os': 'centos7',
        'build-os-base-image-revision': '1612.0',
    })
    

NameError: name 'past_centos' is not defined

In [30]:
for site, image in past_xenial.items():
    image['details'] = glance.image(auth[site], image['id'])
    assert image['details']['name'] == 'CC-Ubuntu16.04'
    glance.image_properties(auth[site], image['id'], add={
        'build-os': 'ubuntu-xenial',
        'build-os-base-image-revision': '20170410',
    })

## Find newest image

In [49]:
for site in auth:
    print(site)
    images = glance.images(auth[site])

    xenial = max((i for i in images if i.get('build-os') == 'ubuntu-xenial'), key=lambda i: i['created_at'])
    centos = max((i for i in images if i.get('build-os') == 'centos7'), key=lambda i: i['created_at'])

    print(xenial['id'], xenial['name'])
    print(centos['id'], centos['name'])
    print()

tacc
00a2d3c7-22cc-4930-b8c0-38eef98a1d97 image-ubuntu-xenial-jenkins-cc-xenial-builder-17
10adbb3b-ccd3-4e23-8623-b8df82073de9 CC-CentOS7

uc
926d0ea3-6e47-40f3-87ba-596f3bbe56cc CC-Ubuntu16.04
388ca9a2-c7d7-49ac-8f66-6b254a6951d4 CC-CentOS7

kvm
c80567e8-12f9-44fc-8335-f62831ee73a6 CC-Ubuntu16.04
053b3438-edb5-4d17-873f-525600b9d9a9 CC-CentOS7



In [59]:
xenial = glance.image(auth['tacc'], '00a2d3c7-22cc-4930-b8c0-38eef98a1d97')
xenial['id'], xenial['name'], xenial['checksum']

('00a2d3c7-22cc-4930-b8c0-38eef98a1d97',
 'image-ubuntu-xenial-jenkins-cc-xenial-builder-17',
 'ad7e4e1542ac60cb2ff18aeaaf45d285')

## Properties to copy

In [53]:
iterprops = iter(set(i) for i in images)
init = next(iterprops)

always_props = set(init)
all_props = set(init)

for props in iterprops:
    always_props &= props
    all_props |= props
    
sometimes_props = all_props - always_props
sometimes_props

{'architecture',
 'base_image_ref',
 'build-os',
 'build-os-base-image-revision',
 'build-repo',
 'build-repo-commit',
 'build-tag',
 'build-variant',
 'build_os',
 'build_os_base_image_revision',
 'build_repo',
 'build_repo_commit',
 'build_tag',
 'build_variant',
 'clean_attempts',
 'description',
 'image_location',
 'image_state',
 'image_type',
 'instance_uuid',
 'kernel_id',
 'owner_id',
 'ramdisk_id',
 'user_id'}

Looks like snapshots on KVM convert hyphens to underscores in the extra properties (`build-os` vs `build_os`). Meh.

In [67]:
extract_extra_properties(xenial)

{'build-os': 'ubuntu-xenial',
 'build-os-base-image-revision': '20171110',
 'build-repo': 'https://github.com/ChameleonCloud/CC-Ubuntu16.04',
 'build-repo-commit': '1b22c495f6c5c7e4c50a8728c05a28dbb7b829ab',
 'build-tag': 'jenkins-cc-xenial-builder-17',
 'build-variant': 'base'}

# Site-to-site Transport

In [6]:
import fabric.api as fapi
import fabric.context_managers as fcm
import fabric.network as fnet

In [7]:
fapi.env['use_ssh_config'] = True
fapi.env['abort_on_prompts'] = True
fapi.env['host_string'] = 'chi01'

In [8]:
fnet.disconnect_all()

In [9]:
def copy_image(source_auth, target_auth, source_image_id, public=None, tempfile=None):
fap

Copy to all the sites!

In [16]:
SOURCE_IMAGE_ID = '9c3bb18b-8171-4f64-9bf0-baef96d8e616'
SOURCE_SITE = 'uc'

targets = [s for s in auth if s != SOURCE_SITE]

new_images = {SOURCE_SITE: glance.image(auth[SOURCE_SITE], SOURCE_IMAGE_ID)}
for target in targets:
    print(f'{SOURCE_SITE} to {target}...', end='')
    new_image = copy_image(auth[SOURCE_SITE], auth[target], SOURCE_IMAGE_ID)
    new_images[target] = new_image
    print('done.')

# copy-to-self to fix owner
new_images[SOURCE_SITE] = copy_image(auth[SOURCE_SITE], auth[SOURCE_SITE], SOURCE_IMAGE_ID)
print('self-copy too.')

uc to tacc...done.
uc to kvm...done.
self-copy too.


In [17]:
new_images

{'kvm': {'build-os': 'ubuntu-trusty',
  'build-os-base-image-revision': '20180206',
  'build-repo': 'https://github.com/ChameleonCloud/CC-Ubuntu16.04',
  'build-repo-commit': 'ea2b696cc7ddc7357f0a95a460c129d4ef553ce7',
  'build-tag': 'jenkins-cc-trusty-builder-9',
  'build-variant': 'base',
  'checksum': '6d2cce71daad2d80d2b1f1d05bddebd6',
  'container_format': 'bare',
  'created_at': '2018-02-09T22:38:15Z',
  'disk_format': 'qcow2',
  'file': '/v2/images/a8eeead0-5991-4a21-8746-02527e5ed085/file',
  'id': 'a8eeead0-5991-4a21-8746-02527e5ed085',
  'min_disk': 0,
  'min_ram': 0,
  'name': 'image-ubuntu-trusty-jenkins-cc-trusty-builder-9',
  'owner': 'openstack',
  'protected': False,
  'schema': '/v2/schemas/image',
  'self': '/v2/images/a8eeead0-5991-4a21-8746-02527e5ed085',
  'size': 527789056,
  'status': 'active',
  'tags': [],
  'updated_at': '2018-02-09T22:38:18Z',
  'virtual_size': None,
  'visibility': 'private'},
 'tacc': {'build-os': 'ubuntu-trusty',
  'build-os-base-image-rev

In [12]:
def single(iterable):
    '''Guarantee there is only one value in *iterable* and returns it'''
    it = iter(iterable)
    try:
        val = next(it)
    except StopIteration:
        raise ValueError('no items in iterable')

    try:
        next(it)
    except StopIteration:
        pass
    else:
        raise ValueError('more than one item in the iterable')
    return val

def single(iterable):
    '''Guarantee there is only one value in *iterable* and returns it'''
    val, = iterable # a bit more opaque...
    return val

def all_equal(iterable):
    it = iter(iterable)
    try:
        val = next(it)
    except StopIteration:
        return True
    for item in it:
        if val != item:
            return False
    return True

assert single('x') == 'x'
assert single(['hey']) == 'hey'
assert single(range(1)) == 0
assert single({'a': 'b'}) == 'a'
assert single({'c'}) == 'c'
for fail in [[], '', {'x', 'y', 'z'}, [1, 2, 3], iter([]), range(0), range(int(1e10)), 'hello']:
    try:
        single(fail)
    except ValueError as e:
        pass

for should_be_true in [[], [1], [2, 2.0], 'aaa', range(0), range(1)]:
    assert all_equal(should_be_true)
for should_be_false in [[1, 2, 3], 'abc', range(2)]:
    assert not all_equal(should_be_false)
for should_not_trap in [(x[3] for x in 'something')]:
    try:
        all_equal(should_not_trap)
    except Exception as e:
        pass
    else:
        assert False

In [18]:
current_name = 'CC-Ubuntu14.04'
current_images = {
    site: single(glance.images(auth[site], {'name': current_name, 'visibility': 'public'}))
    for site in auth
}
assert all_equal(i['checksum'] for i in current_images.values())
current_images

{'kvm': {'checksum': '6b3675d3ec01bfff6b0a2e7346b9c4c2',
  'container_format': 'bare',
  'created_at': '2016-12-23T18:02:51Z',
  'disk_format': 'qcow2',
  'file': '/v2/images/bf5232fb-648b-4a75-a45d-c1b9fec3a02a/file',
  'id': 'bf5232fb-648b-4a75-a45d-c1b9fec3a02a',
  'min_disk': 0,
  'min_ram': 0,
  'name': 'CC-Ubuntu14.04',
  'owner': 'openstack',
  'protected': False,
  'schema': '/v2/schemas/image',
  'self': '/v2/images/bf5232fb-648b-4a75-a45d-c1b9fec3a02a',
  'size': 526712832,
  'status': 'active',
  'tags': [],
  'updated_at': '2016-12-23T20:15:07Z',
  'virtual_size': None,
  'visibility': 'public'},
 'tacc': {'checksum': '6b3675d3ec01bfff6b0a2e7346b9c4c2',
  'container_format': 'bare',
  'created_at': '2016-12-23T17:26:31Z',
  'disk_format': 'qcow2',
  'file': '/v2/images/fc5ef426-bb94-471d-8875-9a662f6c3c92/file',
  'id': 'fc5ef426-bb94-471d-8875-9a662f6c3c92',
  'min_disk': 0,
  'min_ram': 0,
  'name': 'CC-Ubuntu14.04',
  'owner': '570aad8999f7499db99eae22fe9b29bb',
  'prote

# Swap Image Names

In [19]:
print(archival_name(current_images['tacc']))
print(production_name(new_images['tacc']))
assert production_name(new_images['tacc']) == current_images['tacc']['name']

KeyError: 'build-os'

In [22]:
# assert all_equal(x.keys() for x in [auth, new_images, current_images])
# for site in auth:
#     # sanity check (no kill like overkill)
#     current = glance.image(auth[site], current_images[site]['id'])
#     assert current['id'] == current_images[site]['id']
#     assert current['checksum'] == current_images[site]['checksum']

#     new = glance.image(auth[site], new_images[site]['id'])
#     assert new['id'] == new_images[site]['id']
#     assert new['checksum'] == new_images[site]['checksum']
    
#     for k in ['build-os', 'build-variant', 'owner']:
#         assert new[k] == current[k], '{}: "{}" vs "{}"'.format(k, new[k], current[k])
    
#     new_prod_name = production_name(new)
#     assert new_prod_name == current['name']

print('New {} IDs\n---'.format(new_prod_name))
for site in auth:
    # rename old image
    current = current_images[site]
    new = new_images[site]

#     current_archival_name = archival_name(current)
    current_archival_name = 'CC-Ubuntu14.04-20161213'
    new_production_name = production_name(new)
    
    glance.image_properties(auth[site], current['id'], replace={'name': current_archival_name})
    glance.image_properties(auth[site], new['id'], replace={'name': new_production_name, 'visibility': 'public'})
    
    print('{:>5s} : {}'.format(site, new['id']))

New CC-Ubuntu16.04 IDs
---
 tacc : 285fe1dc-b758-4758-a00b-d9c5ab57f76f
   uc : 5a850dc8-1a56-4c54-84ab-c96626f70635
  kvm : a8eeead0-5991-4a21-8746-02527e5ed085


Update IDs in catalog:
* [CC-CentOS7](https://www.chameleoncloud.org/appliances/1/edit/)
  * [-CUDA](https://www.chameleoncloud.org/appliances/37/edit/)
  * [-FPGA](https://www.chameleoncloud.org/appliances/32/edit/)
* [CC-Ubuntu16.04](https://www.chameleoncloud.org/appliances/19/edit/)
  * [-CUDA](https://www.chameleoncloud.org/appliances/40/edit/)
* [CC-Ubuntu14.04](https://www.chameleoncloud.org/appliances/13/edit/)

### A quick 1-off release-candidate (no NVLinks free right now)

In [154]:
cuda_id = 'da2c1bfb-23de-48d0-a748-a4a753739e83'
new_image = copy_image(auth['tacc'], auth['tacc'], cuda_id) # new owner
glance.image_properties(auth['tacc'], new_image['id'], replace={
    'name': 'rc-' + production_name(new_image), 
    'visibility': 'public',
})

[m01-07] run: rm -f /tmp/imgtransfer-d9889f7bb6e5


{'build-os': 'ubuntu-xenial',
 'build-os-base-image-revision': '20171110',
 'build-repo': 'https://github.com/ChameleonCloud/CC-Ubuntu16.04',
 'build-repo-commit': '1b22c495f6c5c7e4c50a8728c05a28dbb7b829ab',
 'build-tag': 'jenkins-cc-xenial-cuda-builder-9',
 'build-variant': 'gpu',
 'checksum': 'af23a781e5ed04947edf074f2fd5d01c',
 'container_format': 'bare',
 'created_at': '2017-11-13T23:25:41Z',
 'disk_format': 'qcow2',
 'file': '/v2/images/97f5871b-43e7-4be5-8554-a84c955abb35/file',
 'id': '97f5871b-43e7-4be5-8554-a84c955abb35',
 'min_disk': 0,
 'min_ram': 0,
 'name': 'rc-CC-Ubuntu16.04-CUDA8',
 'owner': '570aad8999f7499db99eae22fe9b29bb',
 'protected': False,
 'schema': '/v2/schemas/image',
 'self': '/v2/images/97f5871b-43e7-4be5-8554-a84c955abb35',
 'size': 2375090176,
 'status': 'active',
 'tags': [],
 'updated_at': '2017-11-13T23:26:03Z',
 'virtual_size': None,
 'visibility': 'public'}

'a481ee63-19ce-438e-bee9-d244633086db'

In [48]:
copy_image(
    auth['uc'], auth['tacc'], 
    'fec1e6da-f172-484c-92d9-80d27b1c1bdf',
)

{'build-os': 'ubuntu-trusty',
 'build-os-base-image-revision': '20171213',
 'build-repo': 'https://github.com/ChameleonCloud/CC-Ubuntu16.04',
 'build-repo-commit': 'bcd3c4e6fd30459d65f28b83b911c87964c9c577',
 'build-tag': 'jenkins-cc-trusty-builder-7',
 'build-variant': 'base',
 'checksum': '0fdd52a592400e6479752f6375d3bfc4',
 'container_format': 'bare',
 'created_at': '2018-01-06T00:49:53Z',
 'disk_format': 'qcow2',
 'file': '/v2/images/a34d43d5-1f91-4b2c-ad15-cf3e208bccec/file',
 'id': 'a34d43d5-1f91-4b2c-ad15-cf3e208bccec',
 'min_disk': 0,
 'min_ram': 0,
 'name': 'image-ubuntu-trusty-jenkins-cc-trusty-builder-7',
 'owner': '570aad8999f7499db99eae22fe9b29bb',
 'protected': False,
 'schema': '/v2/schemas/image',
 'self': '/v2/images/a34d43d5-1f91-4b2c-ad15-cf3e208bccec',
 'size': 526903808,
 'status': 'active',
 'tags': [],
 'updated_at': '2018-01-06T00:49:58Z',
 'virtual_size': None,
 'visibility': 'private'}

In [47]:
glance.images(auth['uc'], query={'build-os': 'ubuntu-trusty'})

[{'build-os': 'ubuntu-trusty',
  'build-os-base-image-revision': '20171213',
  'build-repo': 'https://github.com/ChameleonCloud/CC-Ubuntu16.04',
  'build-repo-commit': 'bcd3c4e6fd30459d65f28b83b911c87964c9c577',
  'build-tag': 'jenkins-cc-trusty-builder-7',
  'build-variant': 'base',
  'checksum': '0fdd52a592400e6479752f6375d3bfc4',
  'container_format': 'bare',
  'created_at': '2018-01-05T03:43:24Z',
  'disk_format': 'qcow2',
  'file': '/v2/images/fec1e6da-f172-484c-92d9-80d27b1c1bdf/file',
  'id': 'fec1e6da-f172-484c-92d9-80d27b1c1bdf',
  'min_disk': 0,
  'min_ram': 0,
  'name': 'image-ubuntu-trusty-jenkins-cc-trusty-builder-7',
  'owner': '17374548ca3e4a2b83dc877549b665ee',
  'protected': False,
  'schema': '/v2/schemas/image',
  'self': '/v2/images/fec1e6da-f172-484c-92d9-80d27b1c1bdf',
  'size': 526903808,
  'status': 'active',
  'tags': [],
  'updated_at': '2018-01-05T03:43:26Z',
  'virtual_size': None,
  'visibility': 'private'},
 {'build-os': 'ubuntu-trusty',
  'build-os-base-i

In [44]:
glance.image_properties(auth['tacc'], '97f5871b-43e7-4be5-8554-a84c955abb35', replace={
    'name': 'CC-Ubuntu16.04-CUDA8-20171110',
})

{'build-os': 'ubuntu-xenial',
 'build-os-base-image-revision': '20171110',
 'build-repo': 'https://github.com/ChameleonCloud/CC-Ubuntu16.04',
 'build-repo-commit': '1b22c495f6c5c7e4c50a8728c05a28dbb7b829ab',
 'build-tag': 'jenkins-cc-xenial-cuda-builder-9',
 'build-variant': 'gpu',
 'checksum': 'af23a781e5ed04947edf074f2fd5d01c',
 'container_format': 'bare',
 'created_at': '2017-11-13T23:25:41Z',
 'disk_format': 'qcow2',
 'file': '/v2/images/97f5871b-43e7-4be5-8554-a84c955abb35/file',
 'id': '97f5871b-43e7-4be5-8554-a84c955abb35',
 'min_disk': 0,
 'min_ram': 0,
 'name': 'CC-Ubuntu16.04-CUDA8-20171110',
 'owner': '570aad8999f7499db99eae22fe9b29bb',
 'protected': False,
 'schema': '/v2/schemas/image',
 'self': '/v2/images/97f5871b-43e7-4be5-8554-a84c955abb35',
 'size': 2375090176,
 'status': 'active',
 'tags': [],
 'updated_at': '2018-01-06T00:33:15Z',
 'virtual_size': None,
 'visibility': 'public'}

In [46]:
glance.image_properties(auth['tacc'], 'f5f5ef2c-98f8-41ec-8c18-475e0bdbd5e6', replace={
    'name': 'CC-Ubuntu16.04-CUDA8',
    'visibility': 'public',
})

{'build_os': 'ubuntu-xenial',
 'build_os_base_image_revision': '20180105',
 'build_repo': 'https://github.com/ChameleonCloud/CC-Ubuntu16.04',
 'build_repo_commit': 'ea2b696cc7ddc7357f0a95a460c129d4ef553ce7',
 'build_tag': 'jenkins-cc-xenial-cuda-builder-14',
 'build_variant': 'gpu',
 'checksum': 'e9df7f321e391c124b9df11949d8c39e',
 'container_format': 'bare',
 'created_at': '2018-01-06T00:31:17Z',
 'disk_format': 'qcow2',
 'file': '/v2/images/f5f5ef2c-98f8-41ec-8c18-475e0bdbd5e6/file',
 'id': 'f5f5ef2c-98f8-41ec-8c18-475e0bdbd5e6',
 'min_disk': 0,
 'min_ram': 0,
 'name': 'CC-Ubuntu16.04-CUDA8',
 'owner': '570aad8999f7499db99eae22fe9b29bb',
 'protected': False,
 'schema': '/v2/schemas/image',
 'self': '/v2/images/f5f5ef2c-98f8-41ec-8c18-475e0bdbd5e6',
 'size': 2379240960,
 'status': 'active',
 'tags': [],
 'updated_at': '2018-01-06T00:33:59Z',
 'virtual_size': None,
 'visibility': 'public'}